In [1]:
 
ngram_length = 3
dstype = '20NG' 
mname = 'debertaV3'

In [2]:
from colorama import Fore, Style

import os

In [3]:
path = f"/home/bhairavi/om/om3/{dstype}/{ngram_length}grams_{mname}/"
os.makedirs(path, exist_ok=True)

print("Directory created or already exists.")
 

file_path = path + f'{dstype}_{ngram_length}keys.csv' 
print(Fore.YELLOW,"csv_filePATH--->",file_path)


filepath_full = path + f'{dstype}_{ngram_length}top5.csv' 
print(Fore.YELLOW,"QUE_filePATH--->",filepath_full)
 
 
 
modelpath = f"/home/bhairavi/om/om5/{dstype}/{mname}_{dstype}"

 

print(Fore.YELLOW,'modelPATH--->',modelpath)
 


Directory created or already exists.
 csv_filePATH---> /home/bhairavi/om/om3/20NG/1grams_debertaV3/20NG_1keys.csv
 QUE_filePATH---> /home/bhairavi/om/om3/20NG/1grams_debertaV3/20NG_1top5.csv
 modelPATH---> /home/bhairavi/om/om5/20NG/debertaV3_20NG


In [4]:

 
 
import torch  

torch.cuda.empty_cache()  
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BertForSequenceClassification, BertTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
 


# %%
from collections import defaultdict
import string
 
import torch.nn.functional as F
from nltk.util import ngrams

import numpy as np  
  


/home/bhairavi/om/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
 
from datasets import load_dataset

dataset = load_dataset('rungalileo/20_Newsgroups_Fixed')

# %%
dataset


Map: 100%|██████████| 7019/7019 [00:02<00:00, 2509.23 examples/s]


In [ ]:

import pandas as pd
 
test_df = pd.DataFrame(dataset['test'])
train_df = pd.DataFrame(dataset['train'])

# %%
train_df['split'] = 'train'

test_df['split'] = 'test'
 

# %%
df = pd.concat([train_df, test_df], ignore_index=True)

# %%
 
df.shape

# %%
df.dropna(inplace=True)
df.shape

# %%
df.columns

# %%
df['text'][0] , df['label'][0] 

# %%
df['label'].nunique()

 
df.sample(5)
 
df.drop(columns=["id"], inplace=True)
 


# %%
df[0:3]

 
from sklearn.preprocessing import LabelEncoder

# %%
le = LabelEncoder()
df['target'] = le.fit_transform(df['label'])

 
 
numlabel = df['target'].nunique()
numlabel


# %%
df['target'].nunique(), df['label'].nunique()

# %%
df.columns

# %%
numlabel = df['target'].nunique()
numlabel


# %%
df['text'] = df['text'].apply(lambda x: str(x)[:512] if isinstance(x, float) else x[:512])


# %%
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = "cuda"  # the device to load the model onto

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(modelpath)
model = AutoModelForSequenceClassification.from_pretrained(modelpath, num_labels=numlabel)

# Move the model to the specified device
model.to(device)


# %%
max_length = 512

# %%
train_df = df[df['split'] == 'train'].drop(columns=['split'])

test_df = df[df['split'] == 'test'].drop(columns=['split'])
 

# %%
train_df.shape, test_df.shape

# %%
train_val_df = train_df

# %%

# %%
from sklearn.model_selection import StratifiedShuffleSplit

 
sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)  # ~15.79% of remaining data
for train_idx, val_idx in sss_val.split(train_val_df, train_val_df['target']):
    train_df = train_val_df.iloc[train_idx]
    val_df = train_val_df.iloc[val_idx]
 

def tokenize_and_format(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=max_length)
    tokenized_inputs['label'] = list(map(int, examples['target']))
    return tokenized_inputs

# Convert pandas DataFrame to Hugging Face's Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(val_df) 
test_dataset = Dataset.from_pandas(test_df)

# Map the tokenization function across the datasets
train_dataset = train_dataset.map(tokenize_and_format, batched=True,batch_size=16)
eval_dataset = eval_dataset.map(tokenize_and_format, batched=True,batch_size=16) 
test_dataset = test_dataset.map(tokenize_and_format, batched=True,batch_size=16)

In [6]:

from sklearn.metrics import f1_score, precision_score, recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'eval_f1': f1,
        'eval_precision': precision,
        'eval_recall': recall,
    }

 

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Use 'epoch' to evaluate at the end of each epoch
    save_strategy="epoch",  # Also use 'epoch' to save at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model at the end of training based on metric
    metric_for_best_model='f1',  # Define the metric for evaluating the best model
    logging_dir='./logs',
    logging_steps=10,
    report_to=[] 
)
 

trainer = Trainer(
    model=model,
    args=training_args ,  # Here you will need to make sure that the Trainer is set up correctly
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)


/home/bhairavi/om/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
 
io = pd.DataFrame(eval_dataset)
batch_size = 4

# Convert your DataFrame to a list of texts
texts = io['text'].tolist()

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the specified device
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Disable gradient calculations for efficiency
with torch.no_grad():
    # Initialize lists to store predictions
    all_predictions = []
    
    # Process data in batches
    for start in range(0, len(texts), batch_size):
        end = start + batch_size
        batch_texts = texts[start:end]
        
        # Tokenize the batch
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        
        # Move inputs to the same device as model
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Get predictions
        outputs = model(**inputs)
        # Apply softmax to get probabilities from the logits
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        # Get the predicted class indices
        predicted_indices = torch.argmax(probabilities, dim=1)
        
        # Ensure the indices are back on CPU for DataFrame operations
        predicted_indices = predicted_indices.cpu()
        
        # Convert indices to actual labels if labels are strings
        if hasattr(tokenizer, 'get_labels') and callable(tokenizer.get_labels):
            predicted_labels = [tokenizer.get_labels()[idx] for idx in predicted_indices]
        else:
            predicted_labels = predicted_indices.tolist()
        
        # Append batch predictions to the list
        all_predictions.extend(predicted_labels)

io['predicted_label'] = all_predictions
 


In [8]:
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
top_n = 5

In [9]:
io[f'significant_{ngram_length}grams'] = None
io[f'{ngram_length}gram_weights'] = None 

def occlusion(text, model, tokenizer, label, ngram_length, device):
    """
    Perform occlusion on the text and return n-gram importances based on the specified n-gram length.
    """
    inputs = tokenizer(text, return_tensors='pt').to(device)
    original_logits = model(**inputs).logits
    original_probs = F.softmax(original_logits, dim=-1)
    original_prediction = original_probs[0][label].item()
    
    ngram_importances = defaultdict(float)
    words = text.split()
    ngrams_list = list(ngrams(words, ngram_length))
    
    for ngram in ngrams_list:
        occluded_text = " ".join([word if word not in ngram else "[OCCLUDED]" for word in words])
        occluded_inputs = tokenizer(occluded_text, return_tensors='pt').to(device)
        occluded_logits = model(**occluded_inputs).logits
        occluded_probs = F.softmax(occluded_logits, dim=-1)
        occluded_prediction = occluded_probs[0][label].item()
        
        ngram_importances[" ".join(ngram)] = original_prediction - occluded_prediction
        
    return ngram_importances

def aggregate_and_filter_positive_attributions(ngram_attributions, threshold=0):
    significant_ngrams = {
        ngram: value for ngram, value in ngram_attributions.items()
        if value > threshold
    }
    return significant_ngrams

def select_top_ngrams(ngram_importances, top_n=top_n):
    sorted_ngrams = sorted(ngram_importances.items(), key=lambda item: item[1], reverse=True)
    top_ngrams = sorted_ngrams[:top_n]
    return top_ngrams
 
for index, row in io.iterrows():

    if row['target'] == row['predicted_label']:   
                                                     
        ngram_attributions = occlusion(row['text'], model, tokenizer, row['label'], ngram_length, device)
        positive_attributions = aggregate_and_filter_positive_attributions(ngram_attributions)
        top_ngrams = select_top_ngrams(positive_attributions, top_n=top_n)  # Ensure top_n is appropriately set
  
        io.at[index, f'significant_{ngram_length}grams'] = [ngram for ngram, _ in top_ngrams]  # Rename this column to 'significant_ngrams'
      
        io.at[index, f'{ngram_length}gram_weights'] = [weight for _, weight in top_ngrams]  # Rename this column to 'ngram_weights'


In [10]:
io.to_csv(filepath_full,index= False)
                                     
io[0:4]

,text,label,target,__index_level_0__,input_ids,token_type_ids,attention_mask,predicted_label,significant_1grams,1gram_weights
0,\nHad an '83 Alliance for a long time. It was...,7,7,3778,"[1, 6691, 299, 382, 9748, 6599, 270, 266, 455,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7,"['83, used, to, horn, Chevy,]","[0.0008405447006225586, 0.000802159309387207, ..."
1,": >\n: >: From a business point of view, it mi...",13,13,6793,"[1, 877, 2108, 877, 2108, 294, 1007, 266, 460,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13,"[alleiating), >^^^^^^^^^^^^^^^^^^^^^^^, method...","[0.0001329183578491211, 0.00011020898818969727..."
2,"\nJim, please, that's a lame explanation of th...",19,19,9779,"[1, 3644, 261, 811, 261, 272, 280, 268, 266, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",15,None,None
3,"\n\n\n\nHmm, it seems the Little Leaguers didn...",9,9,8990,"[1, 21113, 261, 278, 1015, 262, 3347, 2377, 36...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",9,[],[]


In [11]:
io[f'significant_{ngram_length}grams'][0]

["'83", 'used', 'to', 'horn', 'Chevy,']

In [12]:
io['text'][0], io[f'significant_{ngram_length}grams'][0]

("\nHad an '83 Alliance for a long time.  It was a comfortable but sluggish\ncar.  I got very used to the horn on the stalk, after a couple months worth\nof getting used to it.  After I bought my next car, a Chevy, it took me\nfor-EVER to get used to the horn on the steering wheel again!\n\njim grey\njwg@acd4.acd.com",
 ["'83", 'used', 'to', 'horn', 'Chevy,'])

In [13]:
io['label'] = le.inverse_transform(io['target'])

In [14]:
io= io.dropna()

In [15]:
# label_to_words = io.groupby('label')['significant_words'].apply(lambda words: set().union(*words)).to_dict()
label_to_words = io.groupby('label')[f'significant_{ngram_length}grams'].apply(lambda words: set().union(*words)).to_dict()
# label_to_words = io.groupby('label')['significant_5grams'].apply(lambda words: set().union(*words)).to_dict()
 
print(label_to_words)



{'alt.atheism': {'the', 'observations?', 'equals.', 'punish', 'are', 'kill', 'Scroll', 'what', 'belief.', 'is', 'gods.', 'freedom', 'atheist', 'mornings.', "Qur'an", '(Attempting', 'exist.', 'punishment', 'preacher;', 'Cheers,', "'gnostein", 'cannot', 'religious', 'group...thus', 'religions', 'bobbe@vice.ICO.TEK.COM', 'outside', 'Muslims,', 'ancient', 'atheism."', 'religion', 'BSA"', 'They', 'Arabic.', 'moral', 'god-fearing,', 'Pardon', 'proof,', 'god,', 'Religion,', 'Colon', 'motto', 'make', 'law', 'harm', 'NOT', 'muslim', 'theisms', 'death', 'it', 'definitions', 'atheists', 'idols', 'been', 'defining', '/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\/\\', 'that', 'them', 'correctly,', "don't", 'belief,', 'Jewish', 'imply', 'sex.', 'this.', 'objective', 'proposal', 'OK', 'sexual', 'Sahara.', 'Islam', 'It', 'eternal', 'which', 'waste', 'also', '"Atheism', 'not', '"theory"', 'agree,', 'You', 'No,', 'religion.', 'somone', 'has', 'intole

In [16]:


# # %%
 
# label_to_words_and_weights = {}

# for label, group in io.groupby('label'):
#     word_weights_dict = {}
#     for index, row in group.iterrows():
#         words = row[f'significant_{ngram_length}grams']
#         weights = row[f'{ngram_length}gram_weights']
#         for word, weight in zip(words, weights):
#             if word in word_weights_dict:
#                 word_weights_dict[word] += weight
#             else:
#                 word_weights_dict[word] = weight
#     label_to_words_and_weights[label] = word_weights_dict

# # Display the dictionary with labels, words, and their aggregated weights
# print(label_to_words_and_weights)

label_to_words_and_weights = {}

for label, group in io.groupby('label'):
    word_weights_dict = {}
    for index, row in group.iterrows():
        words = row[f'significant_{ngram_length}grams']
        weights = row[f'{ngram_length}gram_weights']
        for word, weight in zip(words, weights):
            if word in word_weights_dict:
                # Take the maximum of the existing and current weight
                word_weights_dict[word] = max(word_weights_dict[word], weight)
            else:
                word_weights_dict[word] = weight
    label_to_words_and_weights[label] = word_weights_dict

# Display the dictionary with labels, words, and their maximum weights
print(label_to_words_and_weights)


{'alt.atheism': {'god,': 0.0141143798828125, 'genocide.': 0.005432307720184326, 'race,': 0.004642784595489502, 'examples': 0.001633763313293457, 'OK': 0.0015140771865844727, 'behavior': 0.03865039348602295, 'to': 0.02119314670562744, 'that': 0.12826335430145264, 'a': 0.017734825611114502, 'domestication': 0.01579207181930542, "Qur'an": 0.7794345431029797, 'Arabic.': 0.09746944904327393, 'BSA"': 0.3726009130477905, 'sexual': 0.332840234041214, 'EnviroLeague': 0.17510724067687988, 'Scouting': 0.16068971157073975, 'religious': 0.0997464656829834, 'sex.': 0.02186894416809082, 'sex': 0.02162545919418335, 'proven': 0.006862461566925049, 'proposal': 0.003534555435180664, 'of': 0.189692884683609, 'Humans': 0.019738495349884033, 'you': 0.09459894895553589, 'not': 0.04308062791824341, 'morality': 0.009949445724487305, 'is': 0.027204930782318115, 'subjective,': 0.26805223524570465, 'defining': 0.26557914912700653, 'University': 0.25123976171016693, 'which': 0.15605218708515167, 'pay': 0.128249526

In [17]:
# %%
label_to_words = label_to_words_and_weights
 

In [18]:
# %%
len(label_to_words)

19

In [19]:

# %%

label_to_words

 


{'alt.atheism': {'god,': 0.0141143798828125,
  'genocide.': 0.005432307720184326,
  'race,': 0.004642784595489502,
  'examples': 0.001633763313293457,
  'OK': 0.0015140771865844727,
  'behavior': 0.03865039348602295,
  'to': 0.02119314670562744,
  'that': 0.12826335430145264,
  'a': 0.017734825611114502,
  'domestication': 0.01579207181930542,
  "Qur'an": 0.7794345431029797,
  'Arabic.': 0.09746944904327393,
  'BSA"': 0.3726009130477905,
  'sexual': 0.332840234041214,
  'EnviroLeague': 0.17510724067687988,
  'Scouting': 0.16068971157073975,
  'religious': 0.0997464656829834,
  'sex.': 0.02186894416809082,
  'sex': 0.02162545919418335,
  'proven': 0.006862461566925049,
  'proposal': 0.003534555435180664,
  'of': 0.189692884683609,
  'Humans': 0.019738495349884033,
  'you': 0.09459894895553589,
  'not': 0.04308062791824341,
  'morality': 0.009949445724487305,
  'is': 0.027204930782318115,
  'subjective,': 0.26805223524570465,
  'defining': 0.26557914912700653,
  'University': 0.251239761

In [20]:

test_data = pd.DataFrame(test_dataset)

 

 
from nltk.tokenize import sent_tokenize, word_tokenize

# %%
def split_text(text):
 
    sentences = sent_tokenize(text)  # Tokenize the text into sentences
    third_index = len(sentences) // 2  # Calculate index for one-third
    first_part = ' '.join(sentences[:third_index])  # First third of sentences
    remaining_part = ' '.join(sentences[third_index:])  # Remaining two-thirds
    return first_part, remaining_part
 
# Apply the function to the DataFrame
test_data[['first_half', 'second_half']] = test_data['text'].apply(lambda x: pd.Series(split_text(x)))


# %%
test_data['first_half'][0:4] , test_data['second_half'][0:4]

# %%

 
# %%


(0    I am a little confused on all of the models of...
 1    I'm not familiar at all with the format of the...
 2    \nThey were attacking the Iraqis to drive them...
 3                                                     
 Name: first_half, dtype: object,
 0    I am also curious to\nknow what the book value...
 1    :-(\n\nCould you, would you, please send me yo...
 2    And me thinks if the US had not helped out\nth...
 3    \nI've just spent two solid months arguing tha...
 Name: second_half, dtype: object)

In [21]:

# %%
tdf = test_data[['first_half', 'label']]
tdf = Dataset.from_pandas(tdf)


 
   
def tokenize_function(examples):
    return tokenizer(examples['first_half'], truncation=True, padding="max_length", max_length=512)

tdf = tdf.map(tokenize_function, batched=True)

# Predict using the trained model
output = trainer.predict(tdf)
predictions = np.argmax(output.predictions, axis=1)
labels = output.label_ids 

# Generate the classification report
report = classification_report(
    labels,
    predictions,
    target_names=np.unique(labels).astype(str),  # Convert labels to string if necessary
    digits=4
)
print(Fore.RED +"first half i.e partial info classification report, baseline 1")
print(report)



Map:   0%|          | 0/7019 [00:00<?, ? examples/s]

Map: 100%|██████████| 7019/7019 [00:01<00:00, 4199.92 examples/s]


first half i.e partial info classification report, baseline 1
              precision    recall  f1-score   support

           0     0.3391    0.4759    0.3960       290
           1     0.6374    0.6081    0.6224       370
           2     0.5847    0.5799    0.5823       369
           3     0.5100    0.5455    0.5271       374
           4     0.6117    0.5235    0.5642       361
           5     0.8018    0.7108    0.7536       370
           6     0.7688    0.6957    0.7304       368
           7     0.7850    0.7119    0.7467       354
           8     0.5938    0.5873    0.5905       361
           9     0.8019    0.7077    0.7518       366
          10     0.8517    0.7143    0.7770       378
          11     0.7596    0.6056    0.6739       360
          12     0.5983    0.5640    0.5806       367
          13     0.7977    0.7332    0.7640       371
          14     0.5372    0.6277    0.5789       368
          15     0.3716    0.7658    0.5004       380
          16     0.

In [22]:
# Move model to the specified device
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Set batch size
batch_size = 32  # Adjust based on your memory constraints

# Disable gradient calculations for efficiency
with torch.no_grad():
    texts = test_data['first_half'].tolist()
    
    # Create empty lists to store results
    all_predicted_labels = []
    all_predicted_probs = []

    # Process the data in batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        # Tokenize the batch
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

        # Move inputs to the same device as model
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Get predictions
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        
        # Get the top 3 predicted class indices and their probabilities
        top_probs, top_indices = torch.topk(probabilities, 3, dim=1)

        # Move to CPU for DataFrame operations
        top_indices = top_indices.cpu()
        top_probs = top_probs.cpu()

        # Convert indices to actual labels if labels are strings
        if hasattr(tokenizer, 'get_labels') and callable(tokenizer.get_labels):
            batch_predicted_labels = [[tokenizer.get_labels()[idx] for idx in indices] for indices in top_indices]
        else:
            batch_predicted_labels = top_indices.tolist()

        # Append results to the overall list
        all_predicted_labels.extend(batch_predicted_labels)
        all_predicted_probs.extend(top_probs.tolist())

    # Add the predictions and probabilities to the DataFrame
    test_data['top3_predicted_labels'] = all_predicted_labels
    test_data['top3_predicted_probabilities'] = all_predicted_probs



 
test_data = test_data[['text','label','target','first_half','second_half','top3_predicted_labels']]

 
 

# %%
test_data['top3_predicted_target']  =  test_data['top3_predicted_labels']

# %%
test_data['label'] = le.inverse_transform(test_data['target'])

# Decode 'top3_predicted_labels' - since these are lists, we apply the inverse transform in a vectorized manner
test_data['top3_predicted_labels'] = test_data['top3_predicted_labels'].apply(lambda x: le.inverse_transform(x))

 

# %%

# %%
def map_significant_words(predictions):
    print(predictions)
    for row in predictions:
        print(row)
    return [label_to_words.get(row, []) for row in predictions]
    # return [[result[pred] for pred in row] for row in predictions]

# Applying the mapping function
test_data['significant_words'] = test_data['top3_predicted_labels'].apply(map_significant_words)

# %%
test_data


['comp.sys.mac.hardware' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc']
comp.sys.mac.hardware
comp.sys.ibm.pc.hardware
comp.os.ms-windows.misc
['comp.windows.x' 'comp.graphics' 'comp.os.ms-windows.misc']
comp.windows.x
comp.graphics
comp.os.ms-windows.misc
['talk.politics.mideast' 'talk.politics.misc' 'rec.sport.baseball']
talk.politics.mideast
talk.politics.misc
rec.sport.baseball
['soc.religion.christian' 'sci.space' 'alt.atheism']
soc.religion.christian
sci.space
alt.atheism
['alt.atheism' 'talk.religion.misc' 'soc.religion.christian']
alt.atheism
talk.religion.misc
soc.religion.christian
['talk.politics.misc' 'alt.atheism' 'talk.politics.guns']
talk.politics.misc
alt.atheism
talk.politics.guns
['comp.os.ms-windows.misc' 'comp.sys.ibm.pc.hardware' 'comp.windows.x']
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.windows.x
['comp.windows.x' 'comp.graphics' 'comp.os.ms-windows.misc']
comp.windows.x
comp.graphics
comp.os.ms-windows.misc
['comp.graphics' 'sci.electronics' 

,text,label,target,first_half,second_half,top3_predicted_labels,top3_predicted_target,significant_words
0,I am a little confused on all of the models of...,rec.autos,7,I am a little confused on all of the models of...,I am also curious to\nknow what the book value...,"[comp.sys.mac.hardware, comp.sys.ibm.pc.hardwa...","[4, 3, 2]","[{'of': 0.010323107242584229, 'back': 0.006946..."
1,I'm not familiar at all with the format of the...,comp.windows.x,5,I'm not familiar at all with the format of the...,":-(\n\nCould you, would you, please send me yo...","[comp.windows.x, comp.graphics, comp.os.ms-win...","[5, 1, 2]","[{'X': 0.9798418919090182, 'Silicon': 0.027591..."
2,\nThey were attacking the Iraqis to drive them...,talk.politics.mideast,17,\nThey were attacking the Iraqis to drive them...,And me thinks if the US had not helped out\nth...,"[talk.politics.mideast, talk.politics.misc, re...","[17, 18, 9]","[{'Iran': 0.047576963901519775, 'Kayhan': 0.03..."
3,\nI've just spent two solid months arguing tha...,talk.religion.misc,19,,\nI've just spent two solid months arguing tha...,"[soc.religion.christian, sci.space, alt.atheism]","[15, 14, 0]","[{'Christian': 0.8885193075984716, 'Resurrecti..."
4,"\nElisabeth, let's set the record straight for...",sci.med,13,"\nElisabeth, let's set the record straight for...","Elisabeth, I'm going to quote from Zinsser's M...","[alt.atheism, talk.religion.misc, soc.religion...","[0, 19, 15]","[{'god,': 0.0141143798828125, 'genocide.': 0.0..."
...,...,...,...,...,...,...,...,...
7014,"\n Henry, if I read you correctly, you may b...",sci.space,14,"\n Henry, if I read you correctly, you may b...","If that's the case, let me point out that inte...","[sci.space, comp.graphics, talk.politics.misc]","[14, 1, 18]","[{'fuselage': 0.44062113761901855, 'of': 0.983..."
7015,"about\nthem on\n\nActually, I thought Macs wer...",comp.sys.mac.hardware,4,"about\nthem on\n\nActually, I thought Macs wer...",--\nJim Smyton (smytonj@alleg.edu),"[comp.sys.mac.hardware, comp.sys.ibm.pc.hardwa...","[4, 3, 7]","[{'of': 0.010323107242584229, 'back': 0.006946..."
7016,"I sent a version of this post out a while ago,...",rec.sport.baseball,9,"I sent a version of this post out a while ago,...",But I thought we were discussing the need to s...,"[alt.atheism, soc.religion.christian, talk.rel...","[0, 15, 19]","[{'god,': 0.0141143798828125, 'genocide.': 0.0..."
7017,I have this kit which includes the following :...,misc.forsale,6,,I have this kit which includes the following :...,"[soc.religion.christian, sci.space, alt.atheism]","[15, 14, 0]","[{'Christian': 0.8885193075984716, 'Resurrecti..."


In [23]:

# %%
test_data.to_csv(file_path,index= False)


In [24]:

# %%
df = test_data
 
df['significant_words'][0]

 


[{'of': 0.010323107242584229,
  'back': 0.0069468021392822266,
  'I': 0.004542708396911621,
  'board': 0.0038904547691345215,
  'the': 0.11924731731414795,
  'Powerbooks': 0.031394004821777344,
  'upgrade,': 0.010695159435272217,
  'upgrades': 0.00982677936553955,
  'than': 0.006249606609344482,
  'Mac': 0.8320970120839775,
  'upgrade': 0.025358617305755615,
  'Math-Coprocessor.': 0.0030283331871032715,
  "I'm": 0.002179563045501709,
  'some': 0.0040087103843688965,
  'Duo,': 0.126739501953125,
  ':-)': 0.05781334638595581,
  'ones': 0.045670270919799805,
  'better': 0.04330241680145264,
  'IIsi': 0.3973335027694702,
  'alr': 0.019950151443481445,
  'memory.': 0.019033193588256836,
  'meg': 0.017074108123779297,
  'with': 0.0354999303817749,
  'Apple': 0.6213143914937973,
  'Centris': 0.40217870473861694,
  'postings': 0.01035529375076294,
  'require': 0.007758498191833496,
  'Quadra': 0.5522309252992272,
  'Kodak': 0.0068184733390808105,
  'external?': 0.006521821022033691,
  'MacZone

In [25]:
# %%
df.rename(columns={'significant_words': 'significant_words_weights'}, inplace=True)


In [26]:
# %%
len(df['significant_words_weights'][0])

3

In [27]:

# %%
df['significant_words_weights'][0][0]  

# %%


{'of': 0.010323107242584229,
 'back': 0.0069468021392822266,
 'I': 0.004542708396911621,
 'board': 0.0038904547691345215,
 'the': 0.11924731731414795,
 'Powerbooks': 0.031394004821777344,
 'upgrade,': 0.010695159435272217,
 'upgrades': 0.00982677936553955,
 'than': 0.006249606609344482,
 'Mac': 0.8320970120839775,
 'upgrade': 0.025358617305755615,
 'Math-Coprocessor.': 0.0030283331871032715,
 "I'm": 0.002179563045501709,
 'some': 0.0040087103843688965,
 'Duo,': 0.126739501953125,
 ':-)': 0.05781334638595581,
 'ones': 0.045670270919799805,
 'better': 0.04330241680145264,
 'IIsi': 0.3973335027694702,
 'alr': 0.019950151443481445,
 'memory.': 0.019033193588256836,
 'meg': 0.017074108123779297,
 'with': 0.0354999303817749,
 'Apple': 0.6213143914937973,
 'Centris': 0.40217870473861694,
 'postings': 0.01035529375076294,
 'require': 0.007758498191833496,
 'Quadra': 0.5522309252992272,
 'Kodak': 0.0068184733390808105,
 'external?': 0.006521821022033691,
 'MacZone.)': 0.028378188610076904,
 'in

In [28]:
top_n = 15  # The number of top words you want to select

def filter_top_n_words(list_of_dicts):
    processed_list = []
    
    for index, dct in enumerate(list_of_dicts):
        if isinstance(dct, dict):
            # Sort the items by value in descending order
            sorted_items = sorted(dct.items(), key=lambda item: item[1], reverse=True)
            # Select the top N words
            top_items = sorted_items[:top_n]
            # Extract the keys (words) and add them to the list as a set
            processed_list.append(set(key for key, value in top_items))
        else:
            # Print the index and the problematic element
            print(f"Non-dictionary element at index {index}: {dct}")
            # Optionally, add an empty set or handle differently based on your needs
            processed_list.append(set())
    
    return processed_list

# Apply the function to each element in the DataFrame column
df['significant_words'] = df['significant_words_weights'].apply(filter_top_n_words)


Non-dictionary element at index 1: []
Non-dictionary element at index 2: []
Non-dictionary element at index 2: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 0: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 2: []
Non-dictionary element at index 1: []
Non-dictionary element at index 2: []
Non-dictionary element at index 2: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictionary element at index 1: []
Non-dictiona

In [29]:

# %%
df['significant_words'][0][0]  , df['significant_words'][0][1], df['significant_words'][0][2]
# %%


({'100,',
  'Apple',
  'Centris',
  'IIsi',
  'Mac',
  'MacClassic',
  'MacConnection',
  'Macs',
  'PowerBook',
  'Quadra',
  'Quadra.',
  'cmu.comp.sys.mac:',
  'mac',
  'macMags',
  'macinators'},
 {'FastMicro',
  "HD's",
  'MSDOS/Adaptec/APSI',
  'PC.',
  'SGI)',
  'bus.',
  'card',
  'disk',
  'drive',
  'impedance',
  'info.',
  'joystick,',
  'memmory',
  'modules',
  'printer'},
 {'+----------------------------------------------------------------------------+',
  'Math',
  'Phone',
  'WinQVT/Net',
  'Windows',
  'Windows.',
  'card.',
  'config.sys.',
  'driver',
  'drivers.',
  'fonts',
  'ftp',
  'icon',
  'windows',
  'windows.'})

In [30]:

# %%
df.shape
# %%


(7019, 9)

In [31]:

df['significant_words'][0][0]  , df['significant_words_weights'][0][0] 



({'100,',
  'Apple',
  'Centris',
  'IIsi',
  'Mac',
  'MacClassic',
  'MacConnection',
  'Macs',
  'PowerBook',
  'Quadra',
  'Quadra.',
  'cmu.comp.sys.mac:',
  'mac',
  'macMags',
  'macinators'},
 {'of': 0.010323107242584229,
  'back': 0.0069468021392822266,
  'I': 0.004542708396911621,
  'board': 0.0038904547691345215,
  'the': 0.11924731731414795,
  'Powerbooks': 0.031394004821777344,
  'upgrade,': 0.010695159435272217,
  'upgrades': 0.00982677936553955,
  'than': 0.006249606609344482,
  'Mac': 0.8320970120839775,
  'upgrade': 0.025358617305755615,
  'Math-Coprocessor.': 0.0030283331871032715,
  "I'm": 0.002179563045501709,
  'some': 0.0040087103843688965,
  'Duo,': 0.126739501953125,
  ':-)': 0.05781334638595581,
  'ones': 0.045670270919799805,
  'better': 0.04330241680145264,
  'IIsi': 0.3973335027694702,
  'alr': 0.019950151443481445,
  'memory.': 0.019033193588256836,
  'meg': 0.017074108123779297,
  'with': 0.0354999303817749,
  'Apple': 0.6213143914937973,
  'Centris': 0.40

In [32]:

# %%
# %%
df = df.dropna(ignore_index =True)


In [33]:
# %%
df.to_csv(file_path ,index= False)

# %%
df.columns


Index(['text', 'label', 'target', 'first_half', 'second_half',
       'top3_predicted_labels', 'top3_predicted_target',
       'significant_words_weights', 'significant_words'],
      dtype='object')

In [34]:
# %%
len(df['significant_words'][0]),len(df['significant_words'][0][0])
# %%

(3, 15)